In [ ]:
import fastbook
from fastbook import *
from fastai.vision.widgets import *
import fastai

In [ ]:
search_images_ddg

In [ ]:
fastai.torch_core.default_device(False)

In [ ]:
# search duckduckgo and download images to save locally 

results = search_images_ddg('grizzly bear')
ims = results.attrgot('contentUrl')
len(ims)

In [ ]:
dest = 'images/grizzly.jpg'
download_url(ims[0], dest, show_progress = False)

In [ ]:
# check downloaded image

im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
# download and organize images by folder

bear_types = 'grizzly','black','teddy'
path = Path('bears')

if not path.exists():
    path.mkdir()
    for o in bear_types:
        print(o)
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} bear')
        download_images(dest, urls=results)

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
# check for corrupt images after download

failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
# Create DataLoaders for the downloaded dataset using fastai data block API

bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = bears.dataloaders(path)

In [ ]:
# inspect samples of data + labels
# by default, Resize will crop the image to fit a square shape of the requested size, this can distort the image

dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# an alternative is to use Squish

bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# Resize can also be replaced with RandomResizedCrop, a way to transform the downloaded images to augment the data

bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
# Using RandomResizedCrop with an image size of 224 px, 
# which is fairly standard for image classification, and default aug_transforms: 

bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

In [ ]:
# train

learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# Great tool that plots the samples where the loss is the highest. These errors, representing high loss, can mean:
# - the model was CORRECT without confidence
# - the model was WRONG and highly confidence

interp.plot_top_losses(5, nrows=5)

In [ ]:
# describes model
learn.model

In [ ]:
# export model
learn.export()